# Predicting AFL winners using Machine Learning - Cleaning & Feature Preprocessing

---

<img src='img/sportsbet.png' 
style="height:200px;width:700px;">

---



Using the data provided by Sportsbet and CIKM 2015 challenge. We are going to try and maximize winnnings from bet tippings in any given season of AFL. This involves two steps:

* Part 1: Predicting with a high enough accuracy the probability of the home team winning. 

* Part 2: Calculating what the best percentage of our capital to bet is.



The data for this project is sourced from several websites. The meat of the dataset is provided under the **2015 CIKM & SportsBet AFL Challenge**, with others sections such as historical ladder rankings and historical betting odds coming from 

> [The Official AFL Site](http://www.afl.com.au/ladder)
<img src='img/ladder.png'>

---

> [AFL Tables](https://afltables.com/afl/stats/stats_idx.html)
<img src='img/afltables.png'>

---

>[aussportsbetting.com](http://www.aussportsbetting.com/data/historical-afl-results-and-odds-data/)
<img src='img/aussportsbetting.png'>

Since we are unable to use the game statistics of the current game to predict the outcome ("Duh! Otherwise we'd all be millionaires.") We are going to have to do a lot of feature preprocessing to create predictors for our models. Some important ones that come to mind are:

* Previous year's ladder score

* Head-on-head data

* Current form for the team

* Premiership match?

But first lets import some dependencies

#### Import Dependencies
---

In [103]:
# Imports dependencies
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

plt.style.use('fivethirtyeight')

%matplotlib inline

In [104]:
data = pd.read_csv("data/afl_2017.csv")

In [105]:
data["Date"] = pd.to_datetime(data["Date"])

In [106]:
data["Date"][0].year

2017

In [107]:
data["Season"] = data["Date"].map(lambda x: x.year)

In [108]:
data.rename(columns={'Home Team': 'h_team', 'Away Team': 'a_team', "Play Off Game?": "play_off"}, inplace=True)

In [109]:
data.h_team.unique()

array(['Adelaide', 'Richmond', 'GWS Giants', 'Geelong', 'Port Adelaide',
       'Sydney', 'West Coast', 'Essendon', 'Brisbane', 'Collingwood',
       'Hawthorn', 'Fremantle', 'St Kilda', 'Melbourne', 'Carlton',
       'Gold Coast', 'Western Bulldogs', 'North Melbourne'], dtype=object)

#### Team Names and Season Done
---

In [110]:
data["play_off"] = data["play_off"].map(lambda x: 1 if x == "Y" else 0)

In [111]:
data.drop(["Home Goals", "Home Behinds", "Away Goals", "Away Behinds", "Venue", "Kick Off (local)"], inplace = True, axis = 1)

In [112]:
data["margin"] = data["Home Score"] - data["Away Score"]

In [113]:
data

,Date,h_team,a_team,Home Score,Away Score,play_off,Home Odds,Away Odds,Season,margin
0,2017-09-30,Adelaide,Richmond,60,108,1,1.64,2.26,2017,-48
1,2017-09-23,Richmond,GWS Giants,103,67,1,1.54,2.50,2017,36
2,2017-09-22,Adelaide,Geelong,136,75,1,1.43,2.85,2017,61
3,2017-09-16,GWS Giants,West Coast,125,58,1,1.42,2.88,2017,67
4,2017-09-15,Geelong,Sydney,98,39,1,3.10,1.37,2017,59
5,2017-09-09,Port Adelaide,West Coast,76,78,1,1.47,2.68,2017,-2
6,2017-09-09,Sydney,Essendon,121,56,1,1.16,5.24,2017,65
7,2017-09-08,Geelong,Richmond,40,91,1,1.79,2.01,2017,-51
8,2017-09-07,Adelaide,GWS Giants,84,48,1,1.54,2.48,2017,36
9,2017-08-27,West Coast,Adelaide,100,71,0,1.87,1.93,2017,29
